In [6]:
import pandas as pd
import numpy as np
import pandas_gbq as gbq
import pydata_google_auth
import seaborn as sns
import matplotlib.pylab as plt
import datetime
import sys
sys.path.append('/home3/r05322021/Desktop/FB_hatecrime/Code/python/')
import fbscore

In [7]:
def bigquery_auth():
    SCOPES = [
    'https://www.googleapis.com/auth/cloud-platform',
    'https://www.googleapis.com/auth/drive',]
    
    credentials = pydata_google_auth.get_user_credentials(
    SCOPES,
    # Set auth_local_webserver to True to have a slightly more convienient
    # authorization flow. Note, this doesn't work if you're running from a
    # notebook on a remote sever, such as over SSH or with Google Colab.
    auth_local_webserver=True,)

bigquery_auth()

In [8]:
query = '''
WITH OLD_USER AS(
SELECT user_id
FROM `ntufbdata.user_type.user_entering_type`
WHERE TYPE = 'WHOLE'
), REACTION AS(
(SELECT user_id,
        SPLIT(post_id, '_')[ORDINAL(1)] AS page_id,
        post_id
FROM `ntufbdata.USdata.1000_page_us_user_like_post_201501_to_201611_all`
WHERE TIMESTAMP(post_created_date_CT) >= TIMESTAMP('2015-05-03')
AND TIMESTAMP(post_created_date_CT) < TIMESTAMP('2015-05-31'))
UNION DISTINCT
(SELECT user_id,
        SPLIT(post_id, '_')[ORDINAL(1)] AS page_id,
        post_id
FROM `ntufbdata.USdata.politician_us_user_post_like_all`
WHERE TIMESTAMP(post_created_date_CT) >= TIMESTAMP('2015-05-03')
AND TIMESTAMP(post_created_date_CT) < TIMESTAMP('2015-05-31'))
)

SELECT user_id,
        STRING_AGG(page_id, ',') AS like_pages,
        STRING_AGG(CAST(like_time AS STRING), ',') AS like_times
FROM(
SELECT OLD_USER.user_id,
        REACTION.page_id,
        COUNT(*) AS like_time
FROM OLD_USER
INNER JOIN REACTION ON OLD_USER.user_id = REACTION.user_id
GROUP BY OLD_USER.user_id, REACTION.page_id)
GROUP BY user_id
'''

In [9]:
user_like_pages = gbq.read_gbq(query, project_id='ntufbdata')

Downloading: 100%|██████████| 8263859/8263859 [09:56<00:00, 13851.68rows/s]


,user_id,like_pages,like_times
0,831920153488030,"208421665712,10505090694,89033370735,108571612...","6,1,6,1,1,1,1,2,4,1,1,21,1"
1,897450446943730,"354522044588660,174742062548592,12755955064837...","1,2,2,1,7,1,1,1,1,1,1,2,3,2,1,5,2,5,1,1,1,1,1"
2,927754540576641,"10606591490,353026650079,7040724713,8668072881...","2,11,1,9,1,2,1,1,7,2,2,1,1,1,3,1,28,1,21,1,1,6..."
3,793122347433771,"12795435237,124016042296,104301781300,83032188...","2,4,1,2,2,7,1,1,2,1,1,2,1,1,1,3,2,3,9,1,1,2,18..."
4,580149065416331,"21785951839,86680728811,5550296508,39795225543...","1,1,1,1,1,59,1,1,2,1,1,3,2,3"
